In [1]:
#Import Dependencies
%matplotlib inline
import os
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import json
from pprint import pprint
from datetime import datetime
from datetime import timedelta 
import time
import matplotlib.pyplot as plt

In [2]:
#Variables
initial_date = datetime(2008, 1, 1) 
end_date = (datetime.today() - timedelta(days = 1)).date()
print(end_date)

2020-06-05


In [3]:
executable_path = {'executable_path': 'C:/Users/joaqu/OneDrive/Curso/Proy_final/chromedriver.exe'}
browser = Browser('chrome', **executable_path)

In [4]:
#Banxico URL
url = 'https://www.banxico.org.mx/publicaciones-y-prensa/anuncios-de-las-decisiones-de-politica-monetaria/anuncios-politica-monetaria-t.html'
browser.visit(url)
api_url = "https://www.banxico.org.mx/SieAPIRest/service/v1/series/"
banxico_key = "85c8edeab72128102cdad764f0a2d4532c336d22690a51104b500fb6f077ed4c"

In [5]:
html = browser.html
soup = bs(html, 'html.parser')

In [6]:
links = []
for link in soup.find_all('a'):
    if ('/publicaciones-y-prensa/anuncios-de-las-decisiones-de-politica-monetaria') in link.get('href'):
        partial_link = link.get('href')
        links.append('http://www.banxico.org.mx' + partial_link)
links = links[1:]
#links

In [7]:
dates = []
for date in soup.find_all("td"):
    data = date.get_text()
    dates.append(data)
del dates[1::2]

In [8]:
import re
dates2 = []
for index in range(len(dates)):
    dates2.append(re.sub("\s", "", dates[index])) 


In [9]:
dates3 = []
for index in range(len(dates)):
    dates3.append(re.sub("[\/]", "", dates2[index])) 
#print(dates3)

In [10]:
#name_counter = 0
#for i in links:
    #r = requests.get(i)
   # out_file = open(f'{dates3[name_counter]}.pdf', 'wb')
    #out_file.write(r.content)
   # name_counter += 1

In [11]:
final_dates = []
for date in dates2:
    dates = datetime.strptime(date, "%d/%m/%y")
    if dates > initial_date:
        final_dates.append(datetime.strftime(dates, "%Y-%m-%d"))
final_dates.append(datetime.strftime(end_date, "%Y-%m-%d"))

In [12]:
#MXN Lvl
Series = "SF43788"

fxt = []
for fecha in final_dates:
    url_consulta = [f"{api_url}{Series}/datos/{fecha}/{fecha}?token={banxico_key}"]
    for query in url_consulta: 
        data = requests.get(query).json()
        fxt.append(data["bmx"]["series"][0]["datos"][0]["dato"])
time.sleep(60) #tiempo del API

In [ ]:
Series = "SF43878"

tasa = []
for fecha in final_dates:
    try:
        url_consulta = [f"{api_url}{Series}/datos/{fecha}/{fecha}?token={banxico_key}"]
        for query in url_consulta: 
            data = requests.get(query).json()
            tasa.append(data["bmx"]["series"][0]["datos"][0]["dato"])
    except:
        tasa.append(0)
time.sleep(60) #tiempo del API   
    
        

In [ ]:
Series = "SP30578"

#Total Data
total_query_url = f"{api_url}{Series}/datos/{initial_date}/{end_date}?token={banxico_key}"
total_data = requests.get(total_query_url).json()
inflacion = total_data['bmx']['series'][0]['datos']


In [ ]:
#Match fechas


In [ ]:
Banxico_dict = {"MXN": fxt,  "Tasa": tasa, "Inflacion": inflacion }
                
Banxico_data = pd.DataFrame(Banxico_dict)
Banxico_data["Fecha"] = final_dates
Banxico_data = Banxico_data[['Fecha', 'MXN', "Tasa","Inflacion"]]
Banxico_data = Banxico_data.sort_values('Fecha')
Banxico_data = Banxico_data[Banxico_data.Tasa != 0]
Banxico_data = Banxico_data.reset_index(drop = True)

In [ ]:

Banxico_data['Inflacion'] = 0
#Banxico_data['Crecimiento'] = 0
#Banxico_data['Desempleo'] = 0
Banxico_data['Dif'] = 0
Banxico_data['Tasa'] = Banxico_data['Tasa'].astype(float)
Banxico_data

In [ ]:
for i in range(1, len(Banxico_data) -1 ):
    Banxico_data.loc[i, 'Dif'] = 25 * round((100 * (Banxico_data.loc[i, 'Tasa'] - Banxico_data.loc[i+1, 'Tasa']))/25)

In [ ]:
Banxico_data

In [ ]:
# Assign X (data) and y (target)
X = Banxico_data.drop("Dif", axis=1)
y = Banxico_data["Dif"]
print(X.shape, y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

In [ ]:
 classifier.fit(X_train, y_train)

In [ ]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

In [ ]:
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

In [ ]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)